In [1]:
import s3fs 
import boto3
import zipfile
import os


In [2]:
!aws s3 ls eco-w1/in0/mrlc_zip/

2020-12-26 14:25:49 10086708780 Annual_Herb_1985_1995.zip
2020-12-26 14:28:47 10211619774 Annual_Herb_1996_2006.zip
2020-12-26 14:31:47 10245707594 Annual_Herb_2007_2018.zip
2020-12-26 14:34:44 25895252470 Herb_1985_1995.zip
2020-12-26 14:42:11 25858781544 Herb_1996_2006.zip
2020-12-26 14:50:07 20779156209 Herb_2007_2018.zip


In [3]:
import shutil


In [4]:
s3 = s3fs.S3FileSystem()
bucket_path='eco-w1/in0/'
s3.ls(bucket_path)

['eco-w1/in0/cflux', 'eco-w1/in0/mrlc_zip', 'eco-w1/in0/rapv2']

In [5]:
bucket_path='eco-w1/in0/mrlc_zip'

mrlc_files=s3.ls(bucket_path)

In [6]:
for file_obj in mrlc_files:
    if 'Annual' in file_obj:
        print(file_obj)

eco-w1/in0/mrlc_zip/Annual_Herb_1985_1995.zip
eco-w1/in0/mrlc_zip/Annual_Herb_1996_2006.zip
eco-w1/in0/mrlc_zip/Annual_Herb_2007_2018.zip


In [7]:

def download_s3obj_to_local_file(s3_full_path, local_dir):
    bucket_name = s3_full_path.split('/')[0]
    object_name = '/'.join(s3_full_path.split('/')[1:])
    local_full_path = local_dir + '/' + s3_full_path.split('/')[-1]
    
    s3 = boto3.client('s3')
    s3.download_file(bucket_name, object_name, local_full_path)

In [8]:
import subprocess
def subprocess_cmd(command):
    print ("+++"*20)
    print ("CMD is -- %s" % command)
    print ("+++"*20)
    process = subprocess.Popen(command,stdout=subprocess.PIPE, shell=True)
    proc_stdout = process.communicate()[0].strip()
    stupidBytesObject = proc_stdout
    outStr = (stupidBytesObject.decode("utf-8"))
    print(outStr)
    return(outStr)


def s3sync(src,dst):
        print ("hello from s3sync copying dir " + src)
        print ("hello from s3sync copying to dir " + dst)
        pushcmd = "aws s3 sync %s %s" % (src, dst)
        print (pushcmd)
        subprocess_cmd(pushcmd)

In [9]:
def unzip_me(path_to_zip_file, directory_to_extract_to):
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)

In [10]:
#!rm -fr mrlc_ebs_data/

In [11]:
def unzip_file_obj(s3_full_path, directory_to_extract_to):
        local_full_path = local_dir + '/' + s3_full_path.split('/')[-1]
        unzip_me(local_full_path, directory_to_extract_to)
        print('removing file: ',local_full_path)
        os.remove(local_full_path)

In [12]:
def _mkdir(directory):
    try:
        os.stat(directory)
    except:
        os.mkdir(directory)

In [13]:
def gdal_convert_img_tif(input_img, output_tif):
    cmd = f'gdal_translate -of GTiff {input_img} {output_tif}'
    print(cmd)
    os.system(cmd)

In [14]:
def convert_erdas_img_to_tif(tif_output_dir):
    arr = os.listdir(directory_to_extract_to)
    subdir=arr[0]
    files = os.listdir(f'{directory_to_extract_to}/{subdir}')
    for file in files:
        if file.endswith('.img'):
            full_file = f'{directory_to_extract_to}/{subdir}/{file}'
            print(full_file)
            input_img = full_file
            tif_file=file.replace('.img', '.tif')
            output_tif=f'{tif_output_dir}/{tif_file}'
            gdal_convert_img_tif(input_img, output_tif)
    shutil.rmtree(f'{directory_to_extract_to}/{subdir}')

In [15]:
from loganLib.cog_func import cog_create_from_tif
help(cog_create_from_tif)

Help on function cog_create_from_tif in module loganLib.cog_func:

cog_create_from_tif(src_tif, dst_cog)



In [16]:
def convert_tif_cog_single(tif_input, cog_output):
    print(tif_input,cog_output)
    cog_create_from_tif(tif_input,cog_output)

In [17]:
def convert_tif_cog(tif_output_dir, cog_output_dir):
    files = os.listdir(tif_output_dir)
    for file in files:
        if(file.endswith('.tif')):
            print(file)
            tif_input = f'{tif_output_dir}/{file}'
            cog_output = f'{cog_output_dir}/{file}'
            convert_tif_cog_single(tif_input, cog_output)
            os.remove(tif_input)
    

In [18]:
local_dir = 'mrlc_ebs_data'
directory_to_extract_to = 'mrlc_ebs_data/mrlc_erdas_img'
tif_output_dir = 'mrlc_ebs_data/mrlc_tif'
cog_output_dir = 'mrlc_ebs_data/mrlc_cog'
s3_cog_output_path = 's3://eco-w1/in1/mrlc_cog/'

In [19]:

_mkdir(local_dir)
_mkdir(directory_to_extract_to)
_mkdir(tif_output_dir)

for file_obj in mrlc_files[2:3]:
    if 'Annual' in file_obj:
        print(file_obj)
        _mkdir(cog_output_dir)
        download_s3obj_to_local_file(file_obj, local_dir)
        unzip_file_obj(file_obj,directory_to_extract_to)
        convert_erdas_img_to_tif(tif_output_dir)
        convert_tif_cog(tif_output_dir, cog_output_dir)
        s3sync(cog_output_dir, s3_cog_output_path)
        shutil.rmtree(cog_output_dir)

shutil.rmtree(local_dir)
#shutil.rmtree(directory_to_extract_to)
#shutil.rmtree(tif_output_dir)


eco-w1/in0/mrlc_zip/Annual_Herb_2007_2018.zip
removing file:  mrlc_ebs_data/Annual_Herb_2007_2018.zip
mrlc_ebs_data/mrlc_erdas_img/Annual_Herb_2007_2018/nlcd_anhb_2016_mos_v1.img
gdal_translate -of GTiff mrlc_ebs_data/mrlc_erdas_img/Annual_Herb_2007_2018/nlcd_anhb_2016_mos_v1.img mrlc_ebs_data/mrlc_tif/nlcd_anhb_2016_mos_v1.tif
mrlc_ebs_data/mrlc_erdas_img/Annual_Herb_2007_2018/nlcd_anhb_2008_mos_v1.img
gdal_translate -of GTiff mrlc_ebs_data/mrlc_erdas_img/Annual_Herb_2007_2018/nlcd_anhb_2008_mos_v1.img mrlc_ebs_data/mrlc_tif/nlcd_anhb_2008_mos_v1.tif
mrlc_ebs_data/mrlc_erdas_img/Annual_Herb_2007_2018/nlcd_anhb_2018_mos_v1.img
gdal_translate -of GTiff mrlc_ebs_data/mrlc_erdas_img/Annual_Herb_2007_2018/nlcd_anhb_2018_mos_v1.img mrlc_ebs_data/mrlc_tif/nlcd_anhb_2018_mos_v1.tif
mrlc_ebs_data/mrlc_erdas_img/Annual_Herb_2007_2018/nlcd_anhb_2009_mos_v1.img
gdal_translate -of GTiff mrlc_ebs_data/mrlc_erdas_img/Annual_Herb_2007_2018/nlcd_anhb_2009_mos_v1.img mrlc_ebs_data/mrlc_tif/nlcd_anhb_20

In [20]:
! aws s3 ls eco-w1/in1/mrlc_cog/ --human

2021-01-31 18:56:49  754.6 MiB nlcd_anhb_1985_mos_v1.tif
2021-01-31 18:56:49  749.2 MiB nlcd_anhb_1986_mos_v1.tif
2021-01-31 18:56:49  752.0 MiB nlcd_anhb_1987_mos_v1.tif
2021-01-31 18:56:49  748.7 MiB nlcd_anhb_1988_mos_v1.tif
2021-01-31 18:56:49  742.8 MiB nlcd_anhb_1989_mos_v1.tif
2021-01-31 18:57:38  736.2 MiB nlcd_anhb_1990_mos_v1.tif
2021-01-31 18:57:42  742.4 MiB nlcd_anhb_1991_mos_v1.tif
2021-01-31 18:57:45  743.4 MiB nlcd_anhb_1992_mos_v1.tif
2021-01-31 18:57:46  742.9 MiB nlcd_anhb_1993_mos_v1.tif
2021-01-31 18:57:48  762.0 MiB nlcd_anhb_1994_mos_v1.tif
2021-01-31 18:58:33  761.7 MiB nlcd_anhb_1995_mos_v1.tif
2021-01-31 23:55:33  764.0 MiB nlcd_anhb_1996_mos_v1.tif
2021-01-31 23:55:33  746.8 MiB nlcd_anhb_1997_mos_v1.tif
2021-01-31 23:55:33  759.0 MiB nlcd_anhb_1998_mos_v1.tif
2021-01-31 23:55:33  764.0 MiB nlcd_anhb_1999_mos_v1.tif
2021-01-31 23:55:33  759.3 MiB nlcd_anhb_2000_mos_v1.tif
2021-01-31 23:56:12  763.7 MiB nlcd_anhb_2001_mos_v1.tif
2021-01-31 23:56:24  743.7 MiB 

In [21]:
!ls -lh 'mrlc_ebs_data/mrlc_tif'
!ls -lh 'mrlc_ebs_data/mrlc_cog'

ls: cannot access 'mrlc_ebs_data/mrlc_tif': No such file or directory
ls: cannot access 'mrlc_ebs_data/mrlc_cog': No such file or directory


In [22]:
! df -h .

Filesystem      Size  Used Avail Use% Mounted on
/dev/nvme0n1p1  427G  237G  191G  56% /


In [23]:
arr = os.listdir(local_dir)
arr

FileNotFoundError: [Errno 2] No such file or directory: 'mrlc_ebs_data'

In [ ]:
arr = os.listdir(directory_to_extract_to)
files = os.listdir(f'{directory_to_extract_to}/{arr[0]}')
for file in files:
    if file.endswith('.img'):
        print(file)

In [ ]:
! df -h .
!ls -lh mrlc_ebs_data/mrlc_erdas_img

In [ ]:
#!rm -fr mrlc_ebs_data/

In [ ]:
! df -h .

In [ ]:
! gdal_translate -of GTiff mrlc_ebs_data/mrlc_erdas_img/Annual_Herb_1985_1995/nlcd_anhb_1992_mos_v1.img mrlc_ebs_data/mrlc_tif/nlcd_anhb_1992_mos_v1.tif

In [ ]:
! rm -fr mrlc_ebs_data/mrlc_erdas_img/Annual_Herb_1985_1995/